<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-Property" data-toc-modified-id="Read-Property-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read Property</a></span></li><li><span><a href="#Distance-Bus-Stop" data-toc-modified-id="Distance-Bus-Stop-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Distance Bus Stop</a></span></li><li><span><a href="#Distance-Train-Station" data-toc-modified-id="Distance-Train-Station-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Distance Train Station</a></span></li><li><span><a href="#Distance-School" data-toc-modified-id="Distance-School-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Distance School</a></span></li><li><span><a href="#Calculate-Routine-Distance-by-openrouteservice" data-toc-modified-id="Calculate-Routine-Distance-by-openrouteservice-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Calculate Routine Distance by openrouteservice</a></span></li></ul></div>

In [21]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Read Property

In [22]:
df_property=pd.read_csv("../data/curated/domain_2.csv")


df_property=df_property[["url","coordinates"]].copy()
df_property["longitude"]=df_property["coordinates"].map(lambda x: eval(x)[1])
df_property["latitude"]=df_property["coordinates"].map(lambda x: eval(x)[0])

# create gdf
gdf_property = gpd.GeoDataFrame(df_property, geometry=gpd.points_from_xy(df_property.longitude, df_property.latitude, crs="EPSG:4326"))
gdf_property['geometry']  = gdf_property['geometry'] .to_crs("EPSG:4326")
gdf_property=gdf_property[["url","geometry"]].copy()
gdf_property=gdf_property.reset_index(drop=True)
gdf_property.head()

,url,geometry
0,https://www.domain.com.au/1202-620-collins-str...,POINT (144.95435 -37.81866)
1,https://www.domain.com.au/1701-620-collins-str...,POINT (144.95435 -37.81866)
2,https://www.domain.com.au/5707b-639-little-lon...,POINT (144.95217 -37.81419)
3,https://www.domain.com.au/1003-225-elizabeth-s...,POINT (144.96232 -37.81323)
4,https://www.domain.com.au/1202-31-spring-stree...,POINT (144.97407 -37.81450)


In [23]:
gdf_property.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Distance Bus Stop

In [24]:
gdf_busstop=gpd.read_file("../data/raw/PTV_data/PTV_METRO_BUS_STOP/PTV_METRO_BUS_STOP.shp")
gdf_busstop=gdf_busstop[['STOP_ID', 'STOP_NAME', 'LATITUDE', 'LONGITUDE', 'geometry']].copy()
gdf_busstop['geometry']  = gdf_busstop['geometry'] .to_crs("EPSG:4326")

gdf_regionalbusstop=gpd.read_file("../data/raw/PTV_data/PTV_REGIONAL_BUS_STOP/PTV_REGIONAL_BUS_STOP.shp")
gdf_regionalbusstop=gdf_regionalbusstop[['STOP_ID', 'STOP_NAME', 'LATITUDE', 'LONGITUDE',  'geometry']].copy()
gdf_regionalbusstop['geometry']  = gdf_regionalbusstop['geometry'] .to_crs("EPSG:4326")

gdf_busstop=pd.concat([gdf_busstop,gdf_regionalbusstop],ignore_index=True)
gdf_busstop.head()

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,geometry
0,10185,Bonnie View Rd/Maroondah Hwy (Croydon North),-37.780920,145.288145,POINT (145.28815 -37.78091)
1,10186,Primrose Rd/Maroondah Hwy (Croydon North),-37.774626,145.292843,POINT (145.29285 -37.77461)
2,10187,Exeter Rd/Maroondah Hwy (Croydon North),-37.770446,145.295459,POINT (145.29546 -37.77043)
3,10188,Maroondah Golf Park/Maroondah Hwy (Chirnside P...,-37.766346,145.301775,POINT (145.30178 -37.76633)
4,10189,Old Melbourne Rd/Maroondah Hwy (Chirnside Park),-37.765276,145.304410,POINT (145.30442 -37.76526)


In [25]:
gdf_busstop.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [26]:
# define function to calculate the top20 nearest locations' coordinates by geopandas

def get_20_nearest_locations(gdf, point):
    dist_series=gdf["geometry"].distance(point)
    top10_idx=dist_series.argsort().values[:20]
    
    top20_cordinates=gdf.iloc[top10_idx]["geometry"].map(lambda p:(p.x,p.y)).values

    return top20_cordinates

get_20_nearest_locations(gdf=gdf_busstop, point=gdf_property["geometry"][1])

array([(144.95351489772105, -37.819311816789444),
       (144.9534158977541, -37.819165836781785),
       (144.95512985798737, -37.81427487686854),
       (144.95208281845862, -37.81480785666372),
       (144.95530301796674, -37.81418784687984),
       (144.95581277787238, -37.81424184691499),
       (144.9511550985912, -37.815070836602),
       (144.95702705770026, -37.81388481699575),
       (144.94912109836338, -37.8204248464962),
       (144.94903289830071, -37.82114385649469),
       (144.95787989760046, -37.81343985705128),
       (144.9610658963954, -37.8194428573062),
       (144.9611098167439, -37.81616280728887),
       (144.960704096913, -37.815254797255506),
       (144.9613020566593, -37.81663485730496),
       (144.95922485738694, -37.813255807142035),
       (144.96037181705063, -37.81451886722826),
       (144.96032789706518, -37.814454877224826),
       (144.96118109678523, -37.81566879729057),
       (144.96180605631926, -37.818959827353794)], dtype=object)

In [27]:
gdf_property["top20_near_busstop"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_busstop, point=x))
gdf_property.head()

KeyboardInterrupt: 

# Distance Train Station

In [ ]:
gdf_trainstation=gpd.read_file("../data/raw/PTV_data/PTV_METRO_TRAIN_STATION/PTV_METRO_TRAIN_STATION.shp")
gdf_trainstation['geometry']  = gdf_trainstation['geometry'] .to_crs("EPSG:4326")
gdf_trainstation.head()

In [ ]:
gdf_property["top20_near_trainstation"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_trainstation, point=x))
gdf_property.head()

# Distance School

+ there are 4 types of schools, we calculate the distance by school type

In [ ]:
df_school = pd.read_excel("../data/raw/SearchResults.xlsx")
df_school.head()

In [ ]:
df_school["Type"].value_counts()

In [ ]:
# create gdf
gdf_school = gpd.GeoDataFrame(df_school, geometry=gpd.points_from_xy(df_school.Longitude, df_school.Latitude, crs="EPSG:4326"))
gdf_school['geometry']  = gdf_school['geometry'] .to_crs("EPSG:4326")
gdf_school.head()

In [ ]:
# primary school
gdf_property["top20_near_primary"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_school[gdf_school["Type"]=='Primary'], point=x))

# Secondary school
gdf_property["top20_near_secondary"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_school[gdf_school["Type"]=='Secondary'], point=x))


# Combined school
gdf_property["top20_near_combined"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_school[gdf_school["Type"]=='Combined'], point=x))


# special school
gdf_property["top20_near_special"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_school[gdf_school["Type"]=='Special'], point=x))
gdf_property.head()

In [ ]:
gdf_property["property_cordinate"]=gdf_property["geometry"].map(lambda p:(p.x,p.y))
gdf_property.head()

# Calculate Routine Distance by openrouteservice

In [ ]:
import openrouteservice as ors

def calculate_route_distance(start_cordinate, end_cordinates):
    client = ors.Client(base_url="http://localhost:8080/ors")
   
    distances=[]
    
    # calculate 20 times, as we have 20 coordinates for each [busstop, trainstation...]
    for i in range(20):
        coordinates = [[start_cordinate[0], start_cordinate[1]],[end_cordinates[i][0], end_cordinates[i][1]]]
        
        try:
            route = client.directions(
                coordinates=coordinates,
                profile='driving-car',
                format='geojson',
                validate=False,
            )
            distances.append(route['features'][0]["properties"]["summary"]["distance"])
        except Exception as err:
            
            # if error, the distance is more than 100000 metres
            distances.append(1000000)
#             print(err)
        
    # 
    return np.nanmin(np.array(distances))

In [ ]:
%%time

gdf_property["dist_nearest_busstop"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_busstop"]), axis=1)

gdf_property["dist_nearest_trainstation"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_trainstation"]), axis=1)


gdf_property["dist_nearest_primary_school"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_primary"]), axis=1)

gdf_property["dist_nearest_secondary_school"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_secondary"]), axis=1)

gdf_property["dist_nearest_combined_school"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_combined"]), axis=1)

gdf_property["dist_nearest_special_school"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_special"]), axis=1)

In [ ]:
df_property=pd.read_csv("../data/curated/domain_2.csv")
df_property_result=pd.merge(df_property, gdf_property[['url','dist_nearest_busstop', 'dist_nearest_trainstation', 
                                                       'dist_nearest_primary_school', 'dist_nearest_secondary_school', 'dist_nearest_combined_school',
                                                       'dist_nearest_special_school']], on="uri",how="left")
df_property_result.head()

In [ ]:
df_property_result.shape

In [ ]:
df_property_result["dist_nearest_primary_school"]=df_property_result.apply(lambda row: 
                                                                           row["dist_nearest_primary_school"] 
                                                                           if row["dist_nearest_primary_school"] <row["dist_nearest_combined_school"] 
                                                                          else row["dist_nearest_combined_school"], axis=1)

df_property_result["dist_nearest_secondary_school"]=df_property_result.apply(lambda row: 
                                                                           row["dist_nearest_secondary_school"] 
                                                                           if row["dist_nearest_secondary_school"] <row["dist_nearest_combined_school"] 
                                                                          else row["dist_nearest_combined_school"], axis=1)

df_property_result=df_property_result.drop(["dist_nearest_combined_school","dist_nearest_special_school"], axis=1)
df_property_result.to_csv("../data/curated/domain_final2.csv", index=False)